In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.style.use('seaborn-white')

import proplot
proplot.rc['figure.facecolor'] = 'white'

import multiprocess as mp
import numpy as np
import pandas as pd
import bioframe
import cooltools
import cooler

from cooltools.api.eigdecomp import eigs_cis

import glob

# Needed for cis2trans calculations:
from cooltools import coverage
import cooler.balance as cbal
from multiprocess import Pool
from operator import add
from cooler.tools import split
chunksize = 2000000

nthreads = 10

In [47]:
# Figure out path to environment's bedgraphtobigwig: 
import sys
env_pkgs_path = [x for x in sys.path if 'site-packages' in x and 'embryonic-chromatin' in x][0]
env_bins_path = '/'.join(env_pkgs_path.split('/')[:env_pkgs_path.split('/').index('lib')]) + '/bin'

### 1. Load the data

In [5]:
### Read input cooler:
resolution = 25_000
PATH = '../data/coolers/danRer11/'
coolpath = f'{PATH}/Wild-Type_11.danrer11-reduced.mapq_30.1000.mcool::/resolutions/{resolution}'
clr = cooler.Cooler(coolpath)

In [12]:
regions = pd.read_csv('../data/genomes/danRer11/danRer11.armsizes.txt')
regions.loc[:, 'name'] = regions.apply(lambda x: f"{x.chrom}:{x.start}-{x.end}", axis=1)
view_df = bioframe.make_viewframe(regions, name_style='ucsc').query('chrom!="chrM"')


### 2. Calculate E1 and cis2trans

In [8]:
bins = clr.bins()[:]

#### Load replication timing, the best (as we'll prove below) correlate of E1

In [14]:
metadata = bioframe.frac_gene_coverage(bins, 'danRer11')

In [35]:
%%bash
# Download liftover chain file: 
wget -q -P ../data/genome/ https://hgdownload.cse.ucsc.edu/goldenpath/danRer10/liftOver/danRer10ToDanRer11.over.chain.gz

# Download Replication Timing data for Danio (Siefert 2017; GSE85713):
mkdir -p ../data/RT/
cd ../data/RT/
wget -q https://ftp.ncbi.nlm.nih.gov/geo/series/GSE85nnn/GSE85713/suppl/GSE85713_pre_MBT_average.txt.gz
wget -q https://ftp.ncbi.nlm.nih.gov/geo/series/GSE85nnn/GSE85713/suppl/GSE85713_Shield_average.txt.gz
wget -q https://ftp.ncbi.nlm.nih.gov/geo/series/GSE85nnn/GSE85713/suppl/GSE85713_Exp1_S_G1_average.txt.gz
wget -q https://ftp.ncbi.nlm.nih.gov/geo/series/GSE85nnn/GSE85713/suppl/GSE85713_Dome_average.txt.gz
wget -q https://ftp.ncbi.nlm.nih.gov/geo/series/GSE85nnn/GSE85713/suppl/GSE85713_Bud_average.txt.gz

gzip -d *.gz

In [36]:
# Figure out path to environment's liftover: 
import sys
env_pkgs_path = [x for x in sys.path if 'site-packages' in x and 'embryonic-chromatin' in x][0]
env_bins_path = '/'.join(env_pkgs_path.split('/')[:env_pkgs_path.split('/').index('lib')]) + '/bin'

In [42]:
%%bash -s {env_bins_path}
cd ../data/RT/
# Liftover to appropriate genome (danRer11):
for pref in GSE85713_Bud_average GSE85713_Dome_average GSE85713_Exp1_S_G1_average GSE85713_pre_MBT_average GSE85713_Shield_average
do
  awk 'BEGIN{prev=0;ch="1"}{if ($1!=ch) prev=0; print "chr"$1, prev, $2, $3; prev=$2; ch=$1}' ${pref}.txt | sed 's/ /\t/g' > ${pref}.txt.bed
  $1/liftOver ${pref}.txt.bed ../genome/danRer10ToDanRer11.over.chain.gz ${pref}.danrer11.txt.bed ${pref}.unmapped.txt.bed
done

Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates


In [43]:
# Calculate binned metadata for a given resolution:
repl_files = glob.glob('../data/RT/*.danrer11.txt.bed')
repl_titles = list(map(lambda x: '_'.join(x.split('_')[1:-1]), repl_files))

for repl_file, repl_title in zip(repl_files, repl_titles):
    print(repl_title)
    table = bioframe.read_table(repl_file)
    table.columns = ['chrom', 'start', 'end', 'value']

    df_overlapped = bioframe.overlap(bins, table, return_index=True, suffixes=['_1', '_2'])\
        .dropna(subset=['index_2'], axis=0)
    df_overlapped.loc[:, 'feature_start'] = df_overlapped.apply(lambda x: 
        x.start_1 if x.start_1>x.start_2 else x.start_2, 
        axis=1)
    df_overlapped.loc[:, 'feature_end'] = df_overlapped.apply(lambda x: 
        x.end_1 if x.end_1<x.end_2 else x.end_2, 
        axis=1)

    # Replication timing normalized by length of the replication domain in the bin
    df_overlapped.loc[:, 'feature_length'] = df_overlapped.feature_end - df_overlapped.feature_start
    df_overlapped.loc[:, 'norm_value'] = df_overlapped.feature_length * df_overlapped.value_2

    df_overlapped = pd.concat( 
        [ 
            df_overlapped.groupby('index_1').aggregate({'chrom_1':'first', 'start_1':'first','end_1':'first'}),
            df_overlapped.groupby('index_1').apply(lambda x: np.sum(x.norm_value)/np.sum(x.feature_length))
        ],
        axis=1)

    df_overlapped.columns = ['chrom', 'start', 'end', repl_title]

    metadata = pd.merge(metadata, df_overlapped, on=['chrom', 'start', 'end'], how='left')
    
metadata.to_csv(f'../data/RT/metadata.{resolution}.csv')

Dome
Shield
Bud
Exp1_S_G1
pre_MBT


#### Calculate E1 and cis2trans

In [44]:
! mkdir -p ../data/EV/

In [45]:
clr_files = glob.glob( f'{PATH}/*.mcool' )
clr_prefs = list(map(lambda x: x.split('.danrer11-reduced')[0].split('/')[-1], clr_files))

In [46]:
def _zero_rabl(arms_regions, chunk, data):
    chrom_ids = chunk['bins']['chrom']
    pixels = chunk['pixels']
    mask = np.zeros(len(pixels['bin1_id']), dtype=int)
    for region in arms_regions:
        lo, hi = clr.extent(region)
        mask[ (pixels['bin1_id'] >= lo) & (pixels['bin1_id'] <= hi) ] += 1
        mask[ (pixels['bin2_id'] >= lo) & (pixels['bin2_id'] <= hi) ] += 1

    data[mask!=2] = 0
    return data

In [48]:
# Calculate eigenvectors for a given resolution
df = metadata.copy()

lst_eigs = []
var_explained = {}
phasing_track = 'Exp1_S_G1'

for label, clr_file in zip(clr_prefs, clr_files):
    print(label)
    
    coolpath = f"{clr_file}::/resolutions/{resolution}"
    clr = cooler.Cooler(coolpath)
    
    cis_eigs = eigs_cis(
        clr, 
        view_df=view_df, 
        n_eigs=3, 
        phasing_track=metadata[['chrom', 'start', 'end', phasing_track]],
        sort_metric='var_explained')
    
    cis_df = cis_eigs[1].copy()
    var_explained[label] = cis_eigs[0].copy()

    n_bins = len(clr.bins())
    weights = clr.bins()['weight'][:].values

    with Pool(nthreads) as pool:

        cis = ( split(clr, chunksize=chunksize, map=pool.imap_unordered)
                .prepare(cbal._init)
                .pipe(_zero_rabl, regions[['chrom', 'start', 'end']].values)
                .pipe(cbal._zero_diags, 2)
                .pipe(cbal._zero_trans)
                .pipe(cbal._marginalize)
                .reduce(add, np.zeros(n_bins)) )

        trans = ( split(clr, chunksize=chunksize, map=pool.imap_unordered)
                .prepare(cbal._init)
                .pipe(_zero_rabl, regions[['chrom', 'start', 'end']].values)
                .pipe(cbal._zero_cis)
                .pipe(cbal._marginalize)
                .reduce(add, np.zeros(n_bins)) )
    
    cis2trs = cis/trans
    cis2trs[cis2trs==0] = np.nan
    cis2trs[cis==0] = np.nan
    
    cis_df.loc[:, 'cis2trans'] = cis2trs
    
    cis_df.loc[:, 'cis'] = cis
    cis_df.loc[:, 'trans'] = trans

    
    columns = ['chrom', 'start', 'end']
    cis_df = cis_df[columns + ['E1', 'E2', 'E3', 'cis', 'trans', 'cis2trans']]
    cis_df.columns = columns + [ f'E1_{label}', f'E2_{label}', f'E3_{label}', 
                                 f'cis_{label}', f'trans_{label}', f'cis2trans_{label}']

    lst_eigs.append(cis_df.copy())
    
    cis_df.to_csv(f'../data/EV/cis-eigs.{label}.{resolution}.RT-phased.csv')
    
    for val in [f'E1_{label}', f'cis_{label}', f'trans_{label}', f'cis2trans_{label}']:
        bioframe.to_bigwig(cis_df, 
                           clr.chromsizes, 
                           f'../data/EV/{val}.{label}.{resolution}.bw', 
                           value_field=val, 
                           path_to_binary=env_bins_path)
        
    cols_multivec = [ f'E1_{label}', f'E2_{label}', f'E3_{label}']
    cis_df.to_csv(f'../data/EV/cis-eigs.{label}.{resolution}.RT-phased.4multivec.tsv', 
                  sep='\t', 
                  index=False, 
                  header=True)
    
    df = pd.merge(df, cis_df, on=columns, how='left')
    
df.to_csv(f'../data/EV/final_table.{resolution}.RT-phased.csv')

embryos_24hpf


/home/agalicina/anaconda3/envs/embryonic-chromatin/lib/python3.10/site-packages/cooltools/lib/checks.py:550: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in track.groupby([track.columns[0]]):
/tmp/ipykernel_17588/3076477707.py:44: RuntimeWarning: divide by zero encountered in divide
  cis2trs = cis/trans
/tmp/ipykernel_17588/3076477707.py:44: RuntimeWarning: invalid value encountered in divide
  cis2trs = cis/trans


ValueError: Length of values (53818) does not match length of index (51839)